# Modules and Cleaning

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
loc = "./Datasets/Movie_Lens_100k(Small)/"
links = pd.read_csv(loc+"links.csv")
movies = pd.read_csv(loc+"movies.csv")
ratings = pd.read_csv(loc+"new_ratings.csv")
tags = pd.read_csv(loc+"tags.csv")

In [2]:
msk = np.random.rand(len(ratings)) < 0.7
ratings_train = ratings[msk]
ratings_train = ratings_train[['userId','movieId','rating']]
user_index = [x-1 for x in ratings_train.userId.values]
movie_index= [x for x in ratings_train.movieId.values]
rates = ratings_train.rating.values
ratings_test = ratings[~msk]

# Variables

In [3]:
feature_len = 10
U = tf.Variable(initial_value=tf.truncated_normal([671,feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,9125]), name='movies')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])
R = tf.gather(result_flatten, user_index * tf.shape(result)[1] + 
              movie_index, name='extracting_user_rate')

# Cost Function

In [4]:
diff_op = tf.subtract(R, rates, name='trainig_diff')
diff_op_squared = tf.abs(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

# Regularization

In [5]:
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.abs(P, name='movie_abs'), name='movie_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')
cost = tf.add(base_cost, regularizer)

In [6]:
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

# Running Session and Predicting

In [7]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
for i in range(1000):
    sess.run(training_step)

In [8]:
u, p, r = ratings[['userId', 'movieId', 'rating']].values[178]
rhat = tf.gather(tf.gather(result, int(u-1)),int(p))
print("rating for user " + str(u) + " for item " + str(p) + " is " + str(r) + " and our prediction is: " + str(sess.run(rhat)))

rating for user 4.0 for item 537.0 is 4.0 and our prediction is: 4.24103


# Accuracy

In [9]:
user_index_test = [x-1 for x in ratings_test.userId.values]
movie_index_test = [x for x in ratings_test.movieId.values]
rates_test = ratings_test.rating.values

R_test = tf.gather(result_flatten, user_index_test * tf.shape(result)[1] + movie_index_test, name='extracting_user_rate_test')
diff_op_test = tf.subtract(R_test, rates_test, name='test_diff')
diff_op_squared_test = tf.abs(diff_op, name="squared_difference_test")

cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_squared_test, name="squared_difference_test"), name="sum_squared_error_test"),
                   ratings_test.shape[0] * 2, name="average_error")
print(sess.run(cost_test))

0.614734
